# mvtec_anomaly_dataloader

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')
path = '/content/gdrive/Othercomputers/내 노트북/Desktop/Dacon/ComputerVIsion'

import os
os.chdir(path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import cv2
import random
import numpy as np
import pandas as pd
import os.path as osp
import torchvision.transforms as transforms

from glob import glob
from torch.utils.data import Dataset
from torchvision.transforms import functional as F
from PIL import Image


In [4]:
class mvtecDatasetPreprocess:
    def __init__(self, data_dir, mode='train'):
        if mode == 'train':
            self.image_paths = glob(osp.join(data_dir, mode, "*.png"))
            csv_path         = osp.join(data_dir, 'train_df.csv')
            self.labels      = self.read_label_csv(csv_path)

        elif mode == 'test':
            self.image_paths  = glob(osp.join(data_dir, mode, '*.png'))


    def read_label_csv(self, csv_path):
        train_csv = pd.read_csv(csv_path)

        train_labels = train_csv['label']
        label_unique = sorted(np.unique(train_labels))
        label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

        train_labels = [label_unique[k] for k in train_labels]

        return train_labels


In [5]:
class mvtecDataset(Dataset):
    def __init__(self, image_paths, labels, mode='train'):
        self.image_paths = image_paths
        self.labels      = labels
        self.mode        = mode


    def __len__(self):
        return len(self.image_paths)


    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        image = Image.open(image_path).convert('RGB')

        # 데이터 증강
        if self.mode=='train':
            data_transforms = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(45),
                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                transforms.RandomResizedCrop(512),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

            image = data_transforms(image)

        if self.mode=='test':
            pass

        label = self.labels[idx]

        return image, label


    def get_labels(self):
        return self.labels

# mvtec_anomaly_main.py

In [ ]:
!pip install timm
!pip install torchsampler

In [10]:
import sys
import os
import os.path as osp
sys.path.append(osp.join(os.getcwd(), "test"))

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import cv2
import timm
import random
import time
import torch
import torch.nn as nn

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, Subset
from torchsampler.imbalanced import ImbalancedDatasetSampler
#from mvtec_anomaly_dataloader import mvtecDatasetPreprocess, mvtecDataset
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [9]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [11]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)

    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
def debugging_ImbalancedSampler(label_unique, train_loader, device):
    # ImbalancedSampler Debugging ------------------------------------
    invert_label_unique = {value:key for key, value in label_unique.items()}
    df = pd.read_csv("open/train_df.csv")

    for idx, sample_data in enumerate(train_loader):
        if idx == 1: break
        # print(sample_data)""
        x = torch.tensor(sample_data[0], dtype=torch.float32, device=device)
        y = torch.tensor(sample_data[1], dtype=torch.long, device=device)

        label_names = []
        print(f"{'sampled sample_data label':<30} | {'Total Count'}")
        print("-" * 50)
        for sample in y:
            sample = sample.item()
            label_name = invert_label_unique[sample]
            label_names.append(label_name)

            sampled_label_cnt = len(df[df['label'] == label_name])
            print(f"{label_name:<30} | {sampled_label_cnt:>10}")
        print(label_names)
        print(y)
    # ---------------------------------------------------------------------

In [16]:
current_dir = os.getcwd()
print(current_dir)
file_path = os.path.join(current_dir, 'open')
print(file_path)
model_dir = os.path.join(current_dir, 'experiments', 'models')
print(f"Model directory: {model_dir}")

/content/gdrive/Othercomputers/내 노트북/Desktop/Dacon/ComputerVIsion
/content/gdrive/Othercomputers/내 노트북/Desktop/Dacon/ComputerVIsion/open
Model directory: /content/gdrive/Othercomputers/내 노트북/Desktop/Dacon/ComputerVIsion/experiments/models


In [ ]:
# 파라미터 셋팅
batch_size = 32
num_epochs = 120
learning_rate = 1e-3
folds = 5

# GPU 셋팅
if torch.cuda.is_available():
    device = torch.device(f'cuda:{0}')
    print("Use GPU: {} for training".format(0))
else:
    device = torch.device('cpu')
    print('Use CPU')


# data_dir  = osp.join(osp.dirname(__file__), 'open') # NameError: name '__file__' is not defined
# model_dir = osp.join(osp.dirname(__file__), 'experiments', 'models')
# log_dir   = osp.join(osp.dirname(__file__), 'experiments', 'logs')
current_dir = os.getcwd()
data_dir  = os.path.join(current_dir, 'open')
model_dir = os.path.join(current_dir, 'experiments', 'models')
log_dir   = os.path.join(current_dir, 'experiments', 'logs')


# 폴더 존재 점검
if not osp.exists(model_dir):
    os.makedirs(model_dir)
if not osp.exists(log_dir):
    os.makedirs(log_dir)

# 텐서보드 셋팅
writer = SummaryWriter(log_dir)

# 학습 데이터
preprocessor = mvtecDatasetPreprocess(data_dir, mode='train')

# # 테스트 데이터
# test_dataset = mvtecDataset(data_dir, mode='test')
# test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 교차검증
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state = 1)
for fold, (indices_train, indices_valid) in enumerate(kfold.split(preprocessor.image_paths, preprocessor.labels)):
    subset_train_image_paths = [preprocessor.image_paths[idx] for idx in indices_train]
    subset_train_labels      = [preprocessor.labels[idx] for idx in indices_train]

    subset_valid_image_paths = [preprocessor.image_paths[idx] for idx in indices_valid]
    subset_valid_labels      = [preprocessor.labels[idx] for idx in indices_valid]

    train_subset = mvtecDataset(subset_train_image_paths, subset_train_labels)
    valid_subset = mvtecDataset(subset_valid_image_paths, subset_valid_labels)

    train_loader  = DataLoader(train_subset, batch_size=batch_size,
                                sampler=ImbalancedDatasetSampler(train_subset),
                                pin_memory=True,
                                num_workers=8)
    valid_loader  = DataLoader(valid_subset, batch_size=batch_size,
                                shuffle=False,
                                pin_memory=True,
                                num_workers=8)

# 모델 정의
model = Network().to(device)

# 최적화
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

# ------------------------------------ 학습 ------------------------------------
total_train_steps   = torch.tensor(len(train_loader), dtype=torch.int32, device=device)
total_valid_steps   = torch.tensor(len(valid_loader), dtype=torch.int32, device=device)
for epoch in range(num_epochs):
    start = time.time()

    train_loss = torch.zeros(1,     device=device)
    model.train()
    for train_step, (sample_image, sample_label) in enumerate(train_loader):
        # if train_step > 1: break
        start = datetime.now()

        optimizer.zero_grad()
        sample_image = torch.tensor(sample_image, dtype=torch.float32, device=device)
        sample_label = torch.tensor(sample_label, dtype=torch.long,    device=device)

        with torch.cuda.amp.autocast():
            pred = model(sample_image)
            loss = criterion(pred, sample_label)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += torch.div(loss, total_train_steps)

        """
        (a1 + a2 + ... + a100) / 100 = 평균
        a1/100 + a2/100 + .... + a100/100 = 평균
        """

        elapsed_time = datetime.now() - start
        if train_step % 30 == 0:
            print(f"[TRAIN] Elapsed time: {elapsed_time} | Epoch: [{epoch + 1:>4}/{num_epochs}] | step: {train_step+1:>4}/{len(train_loader)} | Train Loss: {train_loss.tolist()[0]:4.4f}")

    # ------------------------------------------------------------------------------

    # ------------------------------------ 검증 ------------------------------------
    valid_loss    = torch.zeros(1,   device=device)
    valid_f1      = 0
    valid_f1_list = []
    model.eval()
    for sample_image, sample_label in valid_loader:
        start = datetime.now()

        sample_image = torch.tensor(sample_image, dtype=torch.float32, device=device)
        sample_label = torch.tensor(sample_label, dtype=torch.long,    device=device)

        with torch.no_grad():
            with torch.cuda.amp.autocast():
                pred = model(sample_image)
                loss = criterion(pred, sample_label)

        valid_loss += torch.div(loss, total_valid_steps)

        valid_pred  = pred.argmax(1).detach().cpu().numpy().tolist()
        sample_label = sample_label.detach().cpu().numpy().tolist()

        valid_f1 += (f1_score(sample_label, valid_pred, average="macro") / len(valid_loader))
        valid_f1_list.append(valid_f1)


    if len(valid_f1_list) > 1 and (valid_f1 > max(valid_f1_list[:-1])):
        torch.save(model.state_dict(), osp.join(model_dir, f'fold_{fold:02d}-epoch_{epoch:03d}.pth'))

    elapsed_time = datetime.now() - start
    time_left = ((num_epochs - epoch+1) * elapsed_time.seconds) / 3600
    print(f"[VALID] Time Left: {time_left:4.2f} | Epoch: [{epoch + 1:>4}/{num_epochs}] | Valid Loss: {valid_loss.tolist()[0]:4.4f} | Valid F1: {valid_f1_list[-1]*100:.4f}%")

    # 텐서보드
    writer.add_scalar('Loss/Train Loss', train_loss.tolist()[0], global_step=epoch)
    writer.add_scalar('Loss/Valid Loss', valid_loss.tolist()[0], global_step=epoch)

Use GPU: 0 for training
[TRAIN] Elapsed time: 0:00:13.737092 | Epoch: [   1/120] | step:    1/107 | Train Loss: 0.0442
[TRAIN] Elapsed time: 0:00:00.702098 | Epoch: [   1/120] | step:   31/107 | Train Loss: 1.3081
[TRAIN] Elapsed time: 0:00:00.686015 | Epoch: [   1/120] | step:   61/107 | Train Loss: 2.5318
[TRAIN] Elapsed time: 0:00:00.540966 | Epoch: [   1/120] | step:   91/107 | Train Loss: 3.7203
[VALID] Time Left: 0.00 | Epoch: [   1/120] | Valid Loss: 4.8768 | Valid F1: 0.3372%
[TRAIN] Elapsed time: 0:00:00.787367 | Epoch: [   2/120] | step:    1/107 | Train Loss: 0.0410
[TRAIN] Elapsed time: 0:00:00.453855 | Epoch: [   2/120] | step:   31/107 | Train Loss: 1.2067
[TRAIN] Elapsed time: 0:00:00.439605 | Epoch: [   2/120] | step:   61/107 | Train Loss: 2.3518
[TRAIN] Elapsed time: 0:00:00.686521 | Epoch: [   2/120] | step:   91/107 | Train Loss: 3.4784
[VALID] Time Left: 0.00 | Epoch: [   2/120] | Valid Loss: 4.8172 | Valid F1: 0.4456%
[TRAIN] Elapsed time: 0:00:00.745673 | Epoch: 